In [ ]:
%load_ext autoreload

In [ ]:
%matplotlib widget

In [ ]:
from pathlib import Path
import flammkuchen as fl
from bouter import EmbeddedExperiment
from tqdm import tqdm 
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

In [ ]:
class LotrExperiment(EmbeddedExperiment):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.scope_config = self["imaging"]["microscope_config"]

        self._dt_imaging = None
        self._resolution = None
        self._n_planes = None
        self._data_shape = None
        self._fn = None
        self._bouts_df = None
        self._traces = None
        self._hdn_indexes = None

    @property
    def fn(self):
        if self._fn is None:
            self._fn = self.scope_config["lightsheet"]["scanning"]["z"]["frequency"]
        return self._fn

    @property
    def dt_imaging(self):
        return 1 / self.fn_imaging
    
    @property
    def dir_name(self):
        return self.root.name
    
    @property
    def bouts_df(self):
        if self._bouts_df is None:
            self._bouts_df = fl.load(self.root / "bouts_df.h5")
        return self._bouts_df
    
    @property
    def traces(self):
        if self._traces is None:
            self._traces = fl.load(self.root / "filtered_traces.h5", "/detr")
        return self._traces
    
    @property
    def hdn_indexes(self):
        if self._hdn_indexes is None:
            self._hdn_indexes = fl.load(self.root / "selected.h5")
        return self._hdn_indexes
    
    @property
    def n_pts(self):
        return self.traces.shape[0]
    
    @property
    def n_rois(self):
        return self.traces.shape[1]
    
    @property
    def has_hdn(self):
        return (self.root / "selected.h5").exists()# self.traces.shape[1]
    
    def find_mirror_dir(self, parent_folder):
        parent_folder = Path(parent_folder)
        matches = list(target_folder.rglob(self.dir_name))
        if len(matches) > 1:
            raise OSError("Multiple folders found for this experiment")
        return matches[0]

In [ ]:
exp = LotrExperiment(file_list[0])

In [ ]:
target_folder = Path("/Users/luigipetrucco/Desktop/test_mirror")

In [ ]:
for f in file_list: 
    exp = LotrExperiment(f)
    if exp["general"]["animal"]["genotype"] != "gad1b:GCaMP6s":
        print(f, exp["general"]["animal"]["genotype"])

In [ ]:
expected=dict(gen="gad1b:GCaMP6s", 
              name="Luigi Petrucco",
              area="aHB",
              setup="Lightsheet")
for f in file_list: 
    exp = LotrExperiment(f)
    entries_dict = dict(gen=exp["general"]["animal"]["genotype"],
                        name = exp["general"]["basic"]["experimenter_name"],
                        area = exp["general"]["basic"]["imaging_area"],
                        setup = exp["general"]["basic"]["setup_name"])
    for k, val in entries_dict.items():
        if val != expected[k]:
            print(f.name, k, ": ", val)

In [ ]:
data = fl.load("/Users/luigipetrucco/Desktop/all_source_data/full_ring/210519_f6/210519_f6_2dvr/data_from_suite2p_unfiltered.h5")

In [ ]:
master_path = Path("/Volumes/Shared/experiments/E0071_lotr/full_ring")
file_list = list(master_path.glob("*[0-9]_f*/*[0-9]_f*"))

In [ ]:
dict_list = []

for f in tqdm(file_list):
    # print(f.name)
    exp = LotrExperiment(f)
    bouts_df = exp.bouts_df
    ring_in_acquisition = (f / "selected.h5").exists()
    exp_dict = dict(fid=f.parent.name,
                    experiment_name=f.name,
                    protocol_name=exp.protocol_name,
                    protocol_version=exp.protocol_version,
                    imaging_area="aHB" if not "ipn" in f.name else "IPN",
                    eyes="eye" in f.name,
                    genotype="gad1b:Gal4;UAS:GCaMP6s",
                    hdn_in_acquisition=exp.has_hdn,
                    imaged_by=exp["general"]["basic"]["experimenter_name"],
                    n_bouts=len(bouts_df),
                    n_turns=np.sum(np.abs(bouts_df["bias"]) > 0.2),
                    n_rois=exp.n_rois,
                    n_hdns=len(exp.hdn_indexes) if exp.has_hdn else None)
    
    dict_list.append(exp_dict)

In [ ]:
exp_df = pd.DataFrame(dict_list)
exp_df = exp_df.set_index("experiment_name")

In [ ]:
np.unique(np.array([f.name.split("_")[2] for f in file_list]))

In [ ]:
from lotr.pca import pca_and_phase

t_start_s = 150

In [ ]:
for f in file_list:
    exp = LotrExperiment(f)
    time_slices_dict = {"natmov": (t_start_s*exp.fn, exp.n_pts - t_start_s*exp.fn), 
                        "clol": (t_start_s*exp.fn, exp.n_pts - t_start_s*exp.fn), 
                        "cwccw": (t_start_s*exp.fn, exp.n_pts // 2),
                        "2dvr": (t_start_s*exp.fn, 2000),
                        "spont": (t_start_s*exp.fn, exp.n_pts - t_start_s*exp.fn),
                        "gainmod": (t_start_s*exp.fn, exp.n_pts - t_start_s*exp.fn)}
    
    if exp.has_hdn:
        traces = exp.traces 
        hdn_indexes = np.array(exp.hdn_indexes)
        t_slice = slice(*[int(t) for t in time_slices_dict[f.name.split("_")[2]]])
        
        pcaed, phase = pca_and_phase(traces[t_slice, hdn_indexes], traces[:, hdn_indexes])

        fig, ax = plt.subplots(1, 1, figsize=(3., 3.))
        ax.plot(pcaed[t_slice, 0], pcaed[t_slice, 1], 
             c=(0.6,)*3, lw=0.5, zorder=-100) # , c=phase, cmap="twilight", lw=3)
        ax.scatter(pcaed[t_slice, 0], pcaed[t_slice, 1], 
                     c=phase[t_slice], lw=0.5, s=5, cmap="twilight",) 
        ax.set_title(f.name)
        plt.axis("equal")
        

In [ ]:
f

In [ ]:
nexexp["stimulus"]["protocol"].keys()

In [ ]:
%autoreload
from lotr import LotrExperiment

In [ ]:
exp.protocol_name

In [ ]:
exp.protocol_version

In [ ]:
dir(exp)